# Decoder model

## This model decodes EEG embeddings from another network to fMRI signal. This other network learns a space shared between EEG and fMRI.

In [34]:
import sys
sys.path.append("..")

import eeg_utils
import fmri_utils
import deep_cross_corr

import numpy as np
from numpy import correlate

import matplotlib.pyplot as plt

import mne
from nilearn.masking import apply_mask, compute_epi_mask

from sklearn.preprocessing import normalize

from scipy.signal import resample

import keras

from keras.initializers import Zeros
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Conv2D, Conv3D, Conv2DTranspose, ZeroPadding2D, Reshape, Flatten, BatchNormalization, LSTM, TimeDistributed, Dense, Lambda, Input, MaxPooling2D, MaxPooling3D 
from keras.optimizers import Adam
from keras.losses import mae
from keras.models import model_from_json

import tensorflow.compat.v1 as tf

import keras.backend as K

import sys

### Load data

In [3]:
mask = fmri_utils.get_population_mask()

#reading data and spliting data into train and test by individuals
eeg_train, bold_train = deep_cross_corr.get_data(list(range(14)), masker=mask)
eeg_test, bold_test = deep_cross_corr.get_data(list(range(14, 16)), masker=mask)

eeg_train = eeg_train.reshape(eeg_train.shape[0], eeg_train.shape[1], eeg_train.shape[2], eeg_train.shape[3], 1)
eeg_test = eeg_test.reshape(eeg_test.shape[0], eeg_test.shape[1], eeg_test.shape[2], eeg_test.shape[3], 1)

bold_train = bold_train.reshape(bold_train.shape[0], bold_train.shape[1], bold_train.shape[2], 1)
bold_test = bold_test.reshape(bold_test.shape[0], bold_test.shape[1], bold_test.shape[2], 1)

/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarni

Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/32/export/20130410320002_Segmentation_bin.vhdr...
Setting channel info structure...
Reading 0 ... 162022  =      0.000 ...   648.088 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)


(16, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/35/export/20130424350002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 197234  =      0.000 ...   788.936 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)


...

(224, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/49/export/20130918490002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 167579  =      0.000 ...   670.316 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


(16, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/50/export/20131003_500002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 168019  =      0.000 ...   672.076 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG', 'O9', 'O10']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


(32, 14164, 20)


### Load EEG model

In [62]:
#LOAD MODEL WEIGHTS - EEG NETWORK
saved_models_path = '../multi_model/'
json_file = open(saved_models_path + 'eeg_demo_0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
eeg_network = model_from_json(loaded_model_json)
# load weights into new model
eeg_network.load_weights(saved_models_path + "eeg_demo_0.h5")

print(eeg_network.summary())

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_31 (Conv3D)           (None, 63, 2, 20, 1)      5         
_________________________________________________________________
batch_normalization_54 (Batc (None, 63, 2, 20, 1)      4         
_________________________________________________________________
conv3d_32 (Conv3D)           (None, 62, 1, 20, 1)      5         
_________________________________________________________________
batch_normalization_55 (Batc (None, 62, 1, 20, 1)      4         
_________________________________________________________________
conv3d_33 (Conv3D)           (None, 61, 1, 20, 1)      3         
_________________________________________________________________
batch_normalization_56 (Batc (None, 61, 1, 20, 1)      4         
_________________________________________________________________
reshape_9 (Reshape)          (None, 61, 20, 1)       

### Load activations from compression network

In [63]:
shared_eeg_train = eeg_network.predict(eeg_train)
shared_eeg_test = eeg_network.predict(eeg_test)

### Build the decoding neural network architecture

In [65]:
input_shape = (None, shared_eeg_train.shape[1], shared_eeg_train.shape[2], 1)

model = Sequential()

model.add(Conv2DTranspose(1, kernel_size=(100, 1),
                          activation='selu', strides=(3,1)))
model.add(BatchNormalization())
model.add(Conv2DTranspose(1, kernel_size=(100, 1), 
                          activation='selu', strides=(50,1)))
model.add(BatchNormalization())
#can we arrange another solution for the padding??
model.add(ZeroPadding2D(padding=(57,0)))

model.build(input_shape)

print(model.summary())

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_45 (Conv2DT (None, 280, 20, 1)        101       
_________________________________________________________________
batch_normalization_5 (Batch (None, 280, 20, 1)        4         
_________________________________________________________________
conv2d_transpose_46 (Conv2DT (None, 14050, 20, 1)      101       
_________________________________________________________________
batch_normalization_6 (Batch (None, 14050, 20, 1)      4         
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 14164, 20, 1)      0         
Total params: 210
Trainable params: 206
Non-trainable params: 4
_________________________________________________________________
None


In [66]:
def correlation(x, y):
    x = K.l2_normalize(x, axis=1)
    y = K.l2_normalize(y, axis=1)

    #flatten because we are dealing with 16x20 matrices
    x = K.batch_flatten(x)
    y = K.batch_flatten(y)

    a = K.batch_dot(x, y, axes=1)

    b = K.batch_dot(x, x, axes=1)
    c = K.batch_dot(y, y, axes=1)

    return 1 - (a / (K.sqrt(b) * K.sqrt(c)))

model.compile(optimizer='adam', loss=correlation)

### Let's train the model

In [68]:
model.fit(shared_eeg_train, bold_train, epochs=100)

Epoch 1/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9944
Epoch 2/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9942
Epoch 3/100
224/224 [==============================] - 2s 9ms/step - loss: 0.9940
Epoch 4/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9938
Epoch 5/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9937
Epoch 6/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9935
Epoch 7/100
224/224 [==============================] - 2s 9ms/step - loss: 0.9933
Epoch 8/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9932
Epoch 9/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9930
Epoch 10/100
224/224 [==============================] - 2s 9ms/step - loss: 0.9929
Epoch 11/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9928
Epoch 12/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9927
Epoc

224/224 [==============================] - 2s 10ms/step - loss: 0.9870
Epoch 99/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9871
Epoch 100/100
224/224 [==============================] - 2s 10ms/step - loss: 0.9870
